In [1]:
from tqdm import tqdm

# bag of wordsの辞書を作成
def create_BoW_dict(file_path):
    BoW_dict = {}
    i = 0
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        for data in data_list:
            data = data.split('\t')
            word_list = data[0].split()
            for word in word_list:
                if word in BoW_dict:
                    continue
                else:
                    BoW_dict[word] = i
                    i += 1
    return BoW_dict

# t=0, e=1, m=2, b=3としてテストデータのベクトルとラベルをそれぞれリストとして作成
def create_vector_and_label(file_path, BoW_dict):
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        vector_list = []
        label_list = []
        for data in tqdm(data_list):
            data = data.split("\t")
            word_list = data[0].split()
            vector = [0] * len(BoW_dict)
            for word in word_list:
                if word in BoW_dict:
                    vector[BoW_dict[word]] += 1
            vector_list.append(vector)
            label_alph = data[1].strip("\n")
            if label_alph == "t":
                label = [1, 0, 0, 0]
            elif label_alph == "e":
                label = [0, 1, 0, 0]
            elif label_alph == "m":
                label = [0, 0, 1, 0]
            elif label_alph == "b":
                label = [0, 0, 0, 1]
            else:
                print("error label")
            label_list.append(label)
    return vector_list, label_list


TRAIN_FEATURE_FILE_PATH = "./news+aggregator/train.feature.txt"

# bag of wordsの辞書型を作成
BoW_dict = create_BoW_dict(TRAIN_FEATURE_FILE_PATH)

In [2]:
import torch

# モデルのクラスが定義されていないとloadできないため
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.l1 = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.l1(x)
        return x

# 52で学習したモデルのロード
model = torch.load('model_weight.pth')

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

correct = 0
total = 0

TEST_FEATURE_FILE_PATH = "./news+aggregator/test.feature.txt"

# test.txtからベクトルリストとラベルリストを作成
x_test_list, y_test_list = create_vector_and_label(TEST_FEATURE_FILE_PATH, BoW_dict)

# データの準備
x_test = torch.Tensor(x_test_list)
y_test = torch.Tensor(y_test_list).long()

# 訓練データのデータローダー
test = TensorDataset(x_test, y_test)
test_loader = DataLoader(test, batch_size=5)

#　testデータの各ラベルの確立を出力
for x, y in test_loader:
    outputs = model(x)
    print(outputs)


100%|██████████| 1335/1335 [00:00<00:00, 7919.23it/s]


tensor([[-1.5069,  4.7528, -2.0053, -1.2506],
        [ 0.0277,  0.1872, -0.7688,  0.6255],
        [-0.7507, -0.1530, -1.3898,  2.3184],
        [-1.5937, -0.9761, -1.6287,  4.2359],
        [-0.6637,  0.1416, -1.0611,  1.6324]], grad_fn=<AddmmBackward0>)
tensor([[-0.2267, -1.6862, -1.7228,  3.6488],
        [-0.0558, -0.4391, -0.1096,  0.6049],
        [-2.2440,  4.2473, -0.9749, -1.0319],
        [ 1.4395, -0.3759, -1.1909,  0.1589],
        [ 3.0122, -1.6326, -1.8623,  0.4255]], grad_fn=<AddmmBackward0>)
tensor([[-0.1978, -0.6009, -1.5733,  2.3880],
        [ 0.7696, -2.9449, -0.5015,  2.7001],
        [-1.1487,  4.6962, -0.9584, -2.5490],
        [-0.1950,  0.0690, -0.9643,  1.1358],
        [ 1.0069, -0.3327, -1.3226,  0.6475]], grad_fn=<AddmmBackward0>)
tensor([[-0.4018, -0.8851, -0.9827,  2.2708],
        [-0.7426,  1.6873, -0.6140, -0.3418],
        [-0.7270,  3.4658, -1.2951, -1.4515],
        [-1.5642,  4.4567, -1.8710, -1.0126],
        [-0.9732,  1.9082, -0.7584, -0.1414]]